# Import Libraries

In [22]:
import time
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import re
import math
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [23]:
location_to_save = "data/final_files/"
large_location_to_save = "data/final_files/large_files/"

# Read Files

Default files.

In [24]:
entire_books = pd.read_csv(location_to_save+"items_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
entire_ratings = pd.read_csv(large_location_to_save+"entire_ratings_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
fairbook_books = pd.read_csv(location_to_save+"fairbook_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
fairbook_ratings = pd.read_csv(location_to_save+"fairbook_ratings_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")

Ratings with removed items that don't have metadata.

In [25]:
entire_ratings_filtered = pd.read_csv(large_location_to_save+"entire_ratings_with_readable_properties_filtered_ISBNs.csv", low_memory=False, index_col=0).fillna("")
fairbook_ratings_filtered = pd.read_csv(location_to_save+"fairbook_ratings_with_readable_properties_filtered_ISBNs.csv", low_memory=False, index_col=0).fillna("")

In [27]:
len(fairbook_ratings_filtered.ISBN.unique()), len(fairbook_books)

(6733, 6733)

Books dataset with correct ISBN (title-author combination).

In [8]:
entire_books_correct_ISBN = pd.read_csv(location_to_save+"items_books_with_readable_properties_correct_ISBN.csv", low_memory=False, index_col=0).fillna("")

ISBN dict.

In [7]:
with open('ISBN_dict.pkl', 'rb') as handle:
        large_ISBN_dict = pkl.load(handle)

# Remove duplicate ISBNs.

Fairbook ratings.

In [12]:
for index, row in fairbook_ratings_filtered.iterrows():
    if row.alt_title!="":
        ISBN = row.ISBN
        ISBN_alt = large_ISBN_dict[ISBN]
        fairbook_ratings_filtered.at[index, "ISBN"] = ISBN_alt

In [13]:
len(fairbook_ratings_filtered), len(fairbook_ratings_filtered.drop_duplicates(["ISBN", "User-ID"]))

(86782, 86356)

In [14]:
fairbook_ratings_filtered_correct = fairbook_ratings_filtered.drop_duplicates(["ISBN", "User-ID"])

Entire ratings.

In [15]:
for index, row in entire_ratings_filtered.iterrows():
    if row.alt_title!="":
        ISBN = row.ISBN
        ISBN_alt = large_ISBN_dict[ISBN]
        entire_ratings_filtered.at[index, "ISBN"] = ISBN_alt

In [16]:
len(entire_ratings_filtered), len(entire_ratings_filtered.drop_duplicates(["ISBN", "User-ID"]))

(1031136, 1021847)

In [17]:
entire_ratings_filtered_correct = entire_ratings_filtered.drop_duplicates(["ISBN", "User-ID"])

### Save!!!

In [18]:
fairbook_ratings_filtered_correct.to_csv(location_to_save + 'fairbook_ratings_with_readable_properties_filtered_correct_ISBNs.csv')
entire_ratings_filtered_correct.to_csv(large_location_to_save + 'entire_ratings_with_readable_properties_filtered_correct_ISBNs.csv')

In [21]:
len(fairbook_ratings_filtered_correct.ISBN.unique())

5504

In [33]:
len(entire_books_correct_ISBN.ISBN_alt.unique())

216657

In [32]:
len(entire_ratings_filtered_correct.ISBN.unique())

222496

In [30]:
len(entire_books)

271360